In [57]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict , Annotated, Literal
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field 
import operator

In [58]:
load_dotenv()

True

In [66]:
model= ChatGroq(model="openai/gpt-oss-120b")

In [67]:
res = model.invoke("Hello, how are you?")
print(res.text)

Hello! I'm doing great, thank you for asking. How can I assist you today?


In [68]:
class SentimentSchema(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description="The sentiment of the review")

In [69]:
class DiagSchema(BaseModel):
    issueType: Literal['UX', 'Performance', 'Bug', 'Other'] = Field(description="The type of issue mentioned in the review")
    tone: Literal['angry', 'sad', 'neutral', 'happy', 'other'] = Field(description="The tone of the review")
    urgency: Literal['low', 'medium', 'high'] = Field(description="The urgency of the issue mentioned in the review")

In [70]:
structured_model = model.with_structured_output(SentimentSchema)
structured_model2 = model.with_structured_output(DiagSchema)

In [71]:
res = structured_model2.invoke("There is something wrong with guidebook my goodbook is missing from product box.")
print(res)

issueType='Bug' tone='neutral' urgency='medium'


In [72]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal['positive', 'negative']
    diagnosis: dict
    response: str

In [73]:
def find_sentiment(state: ReviewState) -> ReviewState:
    prompt = f"Determine if the following review is positive or negative: {state['review']}"
    sentiment = structured_model.invoke(prompt).sentiment
    return {'sentiment': sentiment}

In [74]:
def check_sentiment(state: ReviewState) -> Literal['positive_response', 'run_diagnosis']:

    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

In [75]:
def positive_response(state: ReviewState) :
    prompt = f"Generate a positive response wih warm thanks to the following review. Review: \n{state['review']}"
    res = model.invoke(prompt)
    return {'response': res.text}

In [76]:
def run_diagnosis(state: ReviewState) :
    prompt = f"Analyze the following negative review and provide a diagnosis in terms of issue type, tone, and urgency. Review: \n{state['review']}"
    diag = structured_model2.invoke(prompt)
    
    return {'diagnosis': diag.model_dump()}

In [77]:
def negative_response(state: ReviewState) :
    diagnosis = state['diagnosis']
    prompt = f"""Generate a sympathetic response addressing the issues mentioned in the following review. Review: {state['review']} and '{diagnosis['issueType']}'issue with a '{diagnosis['tone']}' tone and '{diagnosis['urgency']}' urgency."""
    res = model.invoke(prompt)
    return {'response': res.text}

In [ ]:
graph = StateGraph(ReviewState)
graph.add_node('find_sentiment', find_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)



graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_sentiment)
graph.add_edge('positive_response', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)


workflow = graph.compile()
workflow

TypeError: 'function' object is not iterable

In [38]:
initial_state = {'review': "The product was great and I loved it!"}
final_state = workflow.invoke(initial_state)
print(final_state['sentiment'])

positive
